In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.2.1
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
# unzip it
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
#install findshark
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import sys
!pip install pyspark==3.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 kB 14.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853623 sha256=3cbe43af587aa3b91c5c6f6442ef183ee02337b70dab99e02e2c247f1fd4e829
  Stored in directory: /root/.cache/pip/wheels/15/97/bd/52908574a60b5f8e3dc4dc5a0b5be8a59ac20986ee51c2611b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
#import pyshark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.sql.functions import *
#print(puspark._version_)

spark = SparkSession.builder.master("local[*]").appName("TestAnalytics").getOrCreate()
sc=spark.sparkContext

In [ ]:
from pyspark import SparkFiles
import string

url_RomeoJuliet= "https://www.gutenberg.org/cache/epub/1112/pg1112.txt"
url_Hamlet= "https://www.gutenberg.org/files/1524/1524-0.txt"
url_RichardII="https://www.gutenberg.org/cache/epub/1776/pg1776.txt"

spark.sparkContext.addFile(url_Hamlet)
spark.sparkContext.addFile(url_RomeoJuliet)
spark.sparkContext.addFile(url_RichardII)

shakespeare1_rdd=sc.textFile(SparkFiles.get("pg1112.txt"))
shakespeare1_rdd.take(100)

shakespeare2_rdd=sc.textFile(SparkFiles.get("1524-0.txt"))
shakespeare2_rdd.take(100)

shakespeare3_rdd=sc.textFile(SparkFiles.get("pg1776.txt"))
shakespeare3_rdd.take(100)

['The Project Gutenberg eBook of King Richard II',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States,',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '',
 'Title: King Richard II',
 '',
 'Author: William Shakespeare',
 '',
 'Release date: June 1, 1999 [eBook #1776]',
 '                Most recently updated: April 2, 2015',
 '',
 'Language: English',
 '',
 '',
 '',
 '*** START OF THE PROJECT GUTENBERG EBOOK KING RICHARD II ***',
 '',
 'This Etext file is presented by Project Gutenberg, in',
 'cooperation with World Library, Inc., from their Library of the',
 'Future and Shakespeare CDROMS.  Project Gutenberg oft

In [ ]:
import string
def lower_clean_str(str):
  return str.translate(str.maketrans('','',string.punctuation)).lower()

#More verbose
#def lower_clear _str(str):
#  punc='!"#$%\'()*+,./:;<=>?@'
# lowercased_str = str.lower()
# for ch in punc:
#   lowercased_str = lowercased_str.replace(ch,'')
#  return lowercased_str

In [ ]:
shakespeare1_rdd = shakespeare1_rdd.map(lower_clean_str).filter(lambda x: len(x) > 0)
shakespeare1_rdd.take(40)

shakespeare2_rdd = shakespeare2_rdd.map(lower_clean_str).filter(lambda x: len(x) > 0)
shakespeare2_rdd.take(40)

shakespeare3_rdd = shakespeare3_rdd.map(lower_clean_str).filter(lambda x: len(x) > 0)
shakespeare3_rdd.take(40)

['the project gutenberg ebook of king richard ii',
 '    ',
 'this ebook is for the use of anyone anywhere in the united states and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever you may copy it give it away or reuse it under the terms',
 'of the project gutenberg license included with this ebook or online',
 'at wwwgutenbergorg if you are not located in the united states',
 'you will have to check the laws of the country where you are located',
 'before using this ebook',
 'title king richard ii',
 'author william shakespeare',
 'release date june 1 1999 ebook 1776',
 '                most recently updated april 2 2015',
 'language english',
 ' start of the project gutenberg ebook king richard ii ',
 'this etext file is presented by project gutenberg in',
 'cooperation with world library inc from their library of the',
 'future and shakespeare cdroms  project gutenberg often releases',
 'etexts that are not placed in the public domain',
 't

In [ ]:
# Grouping words and counting them

import re

words1_rdd = shakespeare1_rdd.flatMap(lambda line: line.split(' ')).filter(lambda x: re.match('[a-z]+',x))
count1_rdd = words1_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
count1_rdd.take(40)

words2_rdd = shakespeare2_rdd.flatMap(lambda line: line.split(' ')).filter(lambda x: re.match('[a-z]+',x))
count2_rdd = words2_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
count2_rdd.take(40)

words3_rdd = shakespeare3_rdd.flatMap(lambda line: line.split(' ')).filter(lambda x: re.match('[a-z]+',x))
count3_rdd = words3_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
count3_rdd.take(40)

[('project', 109),
 ('gutenberg', 47),
 ('ebook', 13),
 ('of', 749),
 ('king', 259),
 ('this', 249),
 ('is', 318),
 ('use', 25),
 ('anyone', 6),
 ('anywhere', 2),
 ('in', 366),
 ('united', 15),
 ('other', 41),
 ('world', 33),
 ('at', 82),
 ('no', 109),
 ('restrictions', 2),
 ('whatsoever', 2),
 ('may', 84),
 ('give', 38),
 ('away', 30),
 ('reuse', 2),
 ('online', 4),
 ('are', 132),
 ('have', 145),
 ('check', 6),
 ('country', 5),
 ('where', 42),
 ('before', 23),
 ('using', 6),
 ('title', 7),
 ('shakespeare', 18),
 ('june', 2),
 ('april', 1),
 ('language', 2),
 ('start', 3),
 ('presented', 1),
 ('cooperation', 1),
 ('library', 17),
 ('inc', 11)]

In [ ]:
# Removing stopwords

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

def filter_stopwords(word):
    return word not in stopwords

all_documents_rdd = shakespeare1_rdd.union(shakespeare2_rdd).union(shakespeare3_rdd)
wordsall_rdd = all_documents_rdd.flatMap(lambda line: line.split(' ')).filter(lambda x: re.match('[a-z]+',x)).filter(filter_stopwords)
countstopall_rdd = wordsall_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
countstopall_rdd.take(40)

df = spark.createDataFrame(countstopall_rdd, ['word','count'])
df.collect()
!rm -rf "./stopwords_output.csv"
df.write.csv("stopwords_output.csv", header=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
# Grouping words and applying a label on them


document1_words_rdd = words1_rdd.filter(filter_stopwords).map(lambda word: (word,'doc1')).reduceByKey(lambda a, b: a)
document2_words_rdd = words2_rdd.filter(filter_stopwords).map(lambda word: (word,'doc2')).reduceByKey(lambda a, b: a)
document3_words_rdd = words3_rdd.filter(filter_stopwords).map(lambda word: (word,'doc3')).reduceByKey(lambda a, b: a)

documents_labeled_rdd = document1_words_rdd.union(document2_words_rdd).union(document3_words_rdd).distinct()
index_rdd = documents_labeled_rdd.reduceByKey(lambda a, b: a +", "+b).zipWithIndex()
documents2_words_rdd = index_rdd.map(lambda word: (word[1], word[0][0],word[0][1]))


documents2_words_rdd.take(40)

[(0, 'project', 'doc2, doc1, doc3'),
 (1, 'gutenberg', 'doc2, doc1, doc3'),
 (2, 'ebook', 'doc2, doc1, doc3'),
 (3, 'shakespeare', 'doc2, doc1, doc3'),
 (4, 'use', 'doc2, doc1, doc3'),
 (5, 'anyone', 'doc2, doc1, doc3'),
 (6, 'anywhere', 'doc2, doc1, doc3'),
 (7, 'united', 'doc2, doc1, doc3'),
 (8, 'world', 'doc2, doc1, doc3'),
 (9, 'restrictions', 'doc2, doc1, doc3'),
 (10, 'whatsoever', 'doc2, doc1, doc3'),
 (11, 'may', 'doc2, doc1, doc3'),
 (12, 'give', 'doc2, doc1, doc3'),
 (13, 'away', 'doc2, doc1, doc3'),
 (14, 'reuse', 'doc2, doc1, doc3'),
 (15, 'online', 'doc2, doc1, doc3'),
 (16, 'check', 'doc2, doc1, doc3'),
 (17, 'country', 'doc2, doc1, doc3'),
 (18, 'using', 'doc2, doc1, doc3'),
 (19, 'title', 'doc2, doc1, doc3'),
 (20, 'language', 'doc2, doc1, doc3'),
 (21, 'produced', 'doc2, doc1, doc3'),
 (22, 'dianne', 'doc2'),
 (23, 'bean', 'doc2'),
 (24, 'start', 'doc2, doc1, doc3'),
 (25, 'tragedy', 'doc2, doc1'),
 (26, 'prince', 'doc2, doc1, doc3'),
 (27, 'denmark', 'doc2'),
 (28, '